### Traitement des fichiers ``covid``

#### Comptage du nombre de lignes vides à l'intérieur des fichiers ``covid``

In [1]:
import os

relevant_sources = ['lesoir.be', 
                    'lalibre.be', 
                    'rtbf.be', 
                    'rtlinfo.be', 
                    'dhnet.be', 
                    'sudinfo.be', 
                    'lavenir.net', 
                    'lecho.be', 
                    'levif.be'
                   ]

def find_files_with_empty_lines(source):
    emotions_dir_path = f"./data/tmp/{source}/spacy/texts_emotions_len"

    files_with_empty_lines = []

    for filename in os.listdir(emotions_dir_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(emotions_dir_path, filename)
            with open(file_path, "r", encoding="utf-8") as emotions_file:
                lines = emotions_file.readlines()

                # Exclure les fichiers totalement vides ou avec une seule ligne vide
                if len(lines) > 1:
                    for line_number, line in enumerate(lines, start=1):
                        if not line.strip():
                            files_with_empty_lines.append((filename, line_number))

    return files_with_empty_lines

for source in relevant_sources:
    print(f"\nSource: {source}")
    print("----------------")
    files_with_empty_lines = find_files_with_empty_lines(source)
    if files_with_empty_lines:
        for file, line_number in files_with_empty_lines:
            print(f"File: {file}, Line number: {line_number}")
    else:
        print("Aucun fichier avec des lignes vides.\n")



Source: lesoir.be
----------------
Aucun fichier avec des lignes vides.


Source: lalibre.be
----------------
Aucun fichier avec des lignes vides.


Source: rtbf.be
----------------
File: texts_rtbf.be_20210421.txt, Line number: 48

Source: rtlinfo.be
----------------
Aucun fichier avec des lignes vides.


Source: dhnet.be
----------------
Aucun fichier avec des lignes vides.


Source: sudinfo.be
----------------
File: texts_sudinfo.be_20201012.txt, Line number: 82

Source: lavenir.net
----------------
Aucun fichier avec des lignes vides.


Source: lecho.be
----------------
Aucun fichier avec des lignes vides.


Source: levif.be
----------------
Aucun fichier avec des lignes vides.



#### Update des fichiers JSON ``covid``

In [2]:
import os
import json

relevant_sources = ['lesoir.be', 
                    'lalibre.be', 
                    'rtbf.be', 
                    'rtlinfo.be', 
                    'dhnet.be', 
                    'sudinfo.be', 
                    'lavenir.net', 
                    'lecho.be', 
                    'levif.be'
                   ]

def merge_textrazor_emotions(source, date):
    textrazor_file_path = f"./data/tmp/textrazor/{source}/{source}_{date}.json"
    emotions_file_path = f"./data/tmp/{source}/spacy/texts_emotions_len/texts_{source}_{date}.txt"
    output_file_path = f"./data/final_texts/{source}/{source}_{date}.json"

    if os.path.exists(output_file_path):
        return

    if os.path.exists(textrazor_file_path) and os.path.exists(emotions_file_path):
        with open(textrazor_file_path, "r", encoding="utf-8") as textrazor_file:
            textrazor_entries = []
            for line in textrazor_file:
                line = line.strip()
                if not line:
                    # Considérer la ligne vide comme des données de polarité et d'émotions à zéro
                    textrazor_entries.append({"polarity": {"positive": 0, "negative": 0}, "emotions": {}})
                else:
                    try:
                        textrazor_entry = json.loads(line)
                        textrazor_entries.append(textrazor_entry)
                    except json.JSONDecodeError:
                        print(f"Erreur de décodage JSON pour la ligne : {line}")

        with open(emotions_file_path, "r", encoding="utf-8") as emotions_file:
            emotions_entries = []
            for line in emotions_file:
                line = line.strip()
                if not line:
                    # Considérer la ligne vide comme des données de polarité et d'émotions à zéro
                    emotions_entries.append({"polarity": {"positive": 0, "negative": 0}, "emotions": {}})
                else:
                    try:
                        emotions_entry = json.loads(line)
                        emotions_entries.append(emotions_entry)
                    except json.JSONDecodeError:
                        print(f"Erreur de décodage JSON pour la ligne : {line}")

        if len(textrazor_entries) != len(emotions_entries):
            print(f"La date {date} pose problème car le nombre de lignes est différent.")
            return

        with open(output_file_path, "w", encoding="utf-8") as output_file:
            for textrazor_entry, emotions_entry in zip(textrazor_entries, emotions_entries):
                textrazor_entry["polarity"] = emotions_entry.get("polarity", {})
                textrazor_entry["emotions"] = emotions_entry.get("emotions", {})
                textrazor_entry["occurrences"] = emotions_entry.get("occurrences", {})

                output_file.write(json.dumps(textrazor_entry, ensure_ascii=False))
                output_file.write("\n")

for source in relevant_sources:
    print(f"{source}")
    print("----------------")
    final_dir_path = f"./data/final_texts/{source}"
    if not os.path.exists(final_dir_path):
        os.makedirs(final_dir_path)

    textrazor_dir_path = f"./data/tmp/textrazor/{source}"
    for filename in os.listdir(textrazor_dir_path):
        if filename.endswith(".json"):
            date = filename.split("_")[1].split(".")[0]
            merge_textrazor_emotions(source, date)
    print(f"Les fichiers finaux pour la source {source} ont été traités avec succès.\n")

print("La création de tous les fichiers finaux est terminée.")


lesoir.be
----------------
Les fichiers finaux pour la source lesoir.be ont été traités avec succès.

lalibre.be
----------------
Les fichiers finaux pour la source lalibre.be ont été traités avec succès.

rtbf.be
----------------
Les fichiers finaux pour la source rtbf.be ont été traités avec succès.

rtlinfo.be
----------------
Les fichiers finaux pour la source rtlinfo.be ont été traités avec succès.

dhnet.be
----------------
Les fichiers finaux pour la source dhnet.be ont été traités avec succès.

sudinfo.be
----------------
Les fichiers finaux pour la source sudinfo.be ont été traités avec succès.

lavenir.net
----------------
Les fichiers finaux pour la source lavenir.net ont été traités avec succès.

lecho.be
----------------
Les fichiers finaux pour la source lecho.be ont été traités avec succès.

levif.be
----------------
Les fichiers finaux pour la source levif.be ont été traités avec succès.

La création de tous les fichiers finaux est terminée.


### Traitement des fichiers ``non_covid``

#### Comptage du nombre de lignes vides à l'intérieur des fichiers ``non_covid``

In [3]:
import os

relevant_sources = ['lesoir.be', 
                    'lalibre.be', 
                    'rtbf.be', 
                    'rtlinfo.be', 
                    'dhnet.be', 
                    'sudinfo.be', 
                    'lavenir.net', 
                    'lecho.be', 
                    'levif.be'
                   ]

def find_files_with_empty_lines(source):
    emotions_dir_path = f"./data/tmp/{source}/spacy/non_covid/texts_emotions_len"

    files_with_empty_lines = []

    for filename in os.listdir(emotions_dir_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(emotions_dir_path, filename)
            with open(file_path, "r", encoding="utf-8") as emotions_file:
                lines = emotions_file.readlines()

                # Exclure les fichiers totalement vides ou avec une seule ligne vide
                if len(lines) > 1:
                    for line_number, line in enumerate(lines, start=1):
                        if not line.strip():
                            files_with_empty_lines.append((filename, line_number))

    return files_with_empty_lines

for source in relevant_sources:
    print(f"\nSource: {source}")
    print("----------------")
    files_with_empty_lines = find_files_with_empty_lines(source)
    if files_with_empty_lines:
        for file, line_number in files_with_empty_lines:
            print(f"File: {file}, Line number: {line_number}")
    else:
        print("Aucun fichier avec des lignes vides.\n")



Source: lesoir.be
----------------
File: texts_lesoir.be_20200523.txt, Line number: 4
File: texts_lesoir.be_20200723.txt, Line number: 75
File: texts_lesoir.be_20210219.txt, Line number: 20
File: texts_lesoir.be_20210426.txt, Line number: 70

Source: lalibre.be
----------------
File: texts_lalibre.be_20200620.txt, Line number: 70
File: texts_lalibre.be_20200925.txt, Line number: 132
File: texts_lalibre.be_20210628.txt, Line number: 39

Source: rtbf.be
----------------
File: texts_rtbf.be_20200522.txt, Line number: 36
File: texts_rtbf.be_20200522.txt, Line number: 73
File: texts_rtbf.be_20200622.txt, Line number: 18
File: texts_rtbf.be_20200817.txt, Line number: 82
File: texts_rtbf.be_20200824.txt, Line number: 57
File: texts_rtbf.be_20200824.txt, Line number: 59
File: texts_rtbf.be_20200824.txt, Line number: 153
File: texts_rtbf.be_20200918.txt, Line number: 64
File: texts_rtbf.be_20200918.txt, Line number: 72
File: texts_rtbf.be_20200918.txt, Line number: 74
File: texts_rtbf.be_20200

#### Update des fichiers JSON ``non_covid``

In [4]:
import os
import json

relevant_sources = ['lesoir.be', 
                    'lalibre.be', 
                    'rtbf.be', 
                    'rtlinfo.be', 
                    'dhnet.be', 
                    'sudinfo.be', 
                    'lavenir.net', 
                    'lecho.be', 
                    'levif.be'
                   ]

def merge_textrazor_emotions(source, date):
    textrazor_file_path = f"./data/tmp/textrazor/non_covid/{source}/{source}_{date}.json"
    emotions_file_path = f"./data/tmp/{source}/spacy/non_covid/texts_emotions_len/texts_{source}_{date}.txt"
    output_file_path = f"./data/final_texts/non_covid/{source}/{source}_{date}.json"

    if os.path.exists(output_file_path):
        return

    if os.path.exists(textrazor_file_path) and os.path.exists(emotions_file_path):
        with open(textrazor_file_path, "r", encoding="utf-8") as textrazor_file:
            textrazor_entries = []
            for line in textrazor_file:
                line = line.strip()
                if not line:
                    # Considérer la ligne vide comme des données de polarité et d'émotions à zéro
                    textrazor_entries.append({"polarity": {"positive": 0, "negative": 0}, "emotions": {}})
                else:
                    try:
                        textrazor_entry = json.loads(line)
                        textrazor_entries.append(textrazor_entry)
                    except json.JSONDecodeError:
                        print(f"Erreur de décodage JSON pour la ligne : {line}")

        with open(emotions_file_path, "r", encoding="utf-8") as emotions_file:
            emotions_entries = []
            for line in emotions_file:
                line = line.strip()
                if not line:
                    # Considérer la ligne vide comme des données de polarité et d'émotions à zéro
                    emotions_entries.append({"polarity": {"positive": 0, "negative": 0}, "emotions": {}})
                else:
                    try:
                        emotions_entry = json.loads(line)
                        emotions_entries.append(emotions_entry)
                    except json.JSONDecodeError:
                        print(f"Erreur de décodage JSON pour la ligne : {line}")

        if len(textrazor_entries) != len(emotions_entries):
            print(f"La date {date} pose problème car le nombre de lignes est différent.")
            return

        with open(output_file_path, "w", encoding="utf-8") as output_file:
            for textrazor_entry, emotions_entry in zip(textrazor_entries, emotions_entries):
                textrazor_entry["polarity"] = emotions_entry.get("polarity", {})
                textrazor_entry["emotions"] = emotions_entry.get("emotions", {})
                textrazor_entry["occurrences"] = emotions_entry.get("occurrences", {})

                output_file.write(json.dumps(textrazor_entry, ensure_ascii=False))
                output_file.write("\n")

if not os.path.exists("./data/final_texts/non_covid"):
    os.makedirs("./data/final_texts/non_covid")

for source in relevant_sources:
    print(f"{source}")
    print("----------------")
    final_dir_path = f"./data/final_texts/non_covid/{source}"
    if not os.path.exists(final_dir_path):
        os.makedirs(final_dir_path)

    textrazor_dir_path = f"./data/tmp/textrazor/non_covid/{source}"
    for filename in os.listdir(textrazor_dir_path):
        if filename.endswith(".json"):
            date = filename.split("_")[1].split(".")[0]
            merge_textrazor_emotions(source, date)
    print(f"Les fichiers finaux pour la source {source} ont été traités avec succès.\n")

print("La création de tous les fichiers finaux est terminée.")


lesoir.be
----------------
Les fichiers finaux pour la source lesoir.be ont été traités avec succès.

lalibre.be
----------------
Les fichiers finaux pour la source lalibre.be ont été traités avec succès.

rtbf.be
----------------
Les fichiers finaux pour la source rtbf.be ont été traités avec succès.

rtlinfo.be
----------------
Les fichiers finaux pour la source rtlinfo.be ont été traités avec succès.

dhnet.be
----------------
Les fichiers finaux pour la source dhnet.be ont été traités avec succès.

sudinfo.be
----------------
Les fichiers finaux pour la source sudinfo.be ont été traités avec succès.

lavenir.net
----------------
Les fichiers finaux pour la source lavenir.net ont été traités avec succès.

lecho.be
----------------
Les fichiers finaux pour la source lecho.be ont été traités avec succès.

levif.be
----------------
Les fichiers finaux pour la source levif.be ont été traités avec succès.

La création de tous les fichiers finaux est terminée.
